![title](https://hw-media.herworld.com/public/coffee_vs_tea_health_benefits_singapore_-1.jpg)

I was wondering, what the consumer buy before puting coffee in his basket? what if he's a tea person ? is it sugar ? how to analyse or anticipate cross selling or up selling when someone buy coffee or tea ? 
for information : <br>
![](https://storage.googleapis.com/website-production/uploads/2018/11/cross-selling-vs-upselling.png)

So in this kernel i use data about sales from a bakery. <br>
To found an answers to my question, at first, i’ll be doing a little exploratory data analysis (EDA) followed by a Market Basket Analysis (MBA).

[1. Introduction : ](#Intro) <br>
[2. Exploration :](#Exp) <br>
[3. Sales over timer :](#Sale) <br>
[4. Market Basket Analysis :](#MBA) <br>
[5. Conclusion :](#Conc) <br>
[6. References :](#Book) <br>

**The MBA (Market Basket Analysis) :**
> **Market basket analysis scrutinizes the products customers tend to buy together, and uses the information to decide which products should be cross-sold or promoted together. [*](#Book)  **

There are three most popular criteria evaluating the quality or the strength of an association rule: support, confidence and lift.

* ***Support*** is the percentage of transactions containing a particular combination of items relative to the total number of transactions in the data-base.

* ***Confidence*** measures how much the consequent (item) is dependent on the antecedent (item).it is the conditional probability of the consequent given the antecedent, P(B|A). For example, the confidence for the association rule “if bread then ice cream” is 66% since three transactions contain bread (the antecedent) and two among the three transactions also contain ice cream (the consequent). In other words, given that the baskets containing ice cream is selected, there is 66% chance that the same basket also contains ice cream.

      ** Example : ** <br>
      Tong => Sunglass <br>
      * Support=20% ; <br>
      * Confidence=60% ; <br>  

      Above rule says: <br>
      20% transaction show Sunglass is bought with purchase of a Tong
      60% of customers who purchase Sunglass is bought with purchase of a Tong

* ***Lift*** (also called improvement or impact) is a measure to overcome the problems with support and confidence. Lift is said to measure the difference — measured in ratio — between the confidence of a rule and the expected confidence.
    - If the rule had a lift of 1,then A and B are independent and no rule can be derived from them.
    - If the lift is > 1, then A and B are dependent on each other, and the degree of which is given by ift value.
    - If the lift is < 1, then presence of A will have negative effect on B.


**Given a rule "A -> B", A stands for antecedent and B stands for consequent. **

** Support A ==> B ** <br>
Support(A) = (number of transaction in wich the item A appears) / Total number of transactions <br>
P(BA) ==> Support(AB) = (A and B) / Total transactions.


** Confidence A ==> B ** <br>
P(B|A)  ==> Confidence = Support(A and B)/Support(A)


** Lift A ==> B ** <br>
P(B|A)/P(B)  ==> Lift = Support (A and B)/ [Support(B) x Support(A)] </b>


##### *Each criterion has its advantages and disadvantages but in general we keep association rules that have high confidence, high support, and high lift. Association rules with high support are potentially interesting rules. Similarly, rules with high confidence would be interesting rules. Or you may look for association rules with very high or very low lift.*

### 1. *Introduction* :  <a id="Intro"></a> 

In [ ]:
import pandas as pd
import numpy as np 
import seaborn as sns
import matplotlib.pyplot as plt
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
import mlxtend as ml

In [ ]:
basket = pd.read_csv('../input/transactions-from-a-bakery/BreadBasket_DMS.csv')
basket.head(10)

In [ ]:
basket.shape

In [ ]:
basket.isnull().any()

In [ ]:
basket.dtypes

In [ ]:
basket['Item'].unique()

### 2. *Exploration* :  <a id="Exp"></a>

In [ ]:
basket['Date'].value_counts().iloc[:10]

In [ ]:
sns.countplot(x = 'Date', data = basket, order = basket['Date'].value_counts().iloc[:10].index)
plt.xticks(rotation=45)

In [ ]:
basket['Time'].value_counts()

In [ ]:
# plt.figure(figsize=(30,20))
sns.countplot(x = 'Time', data = basket, order = basket['Time'].value_counts().iloc[:25].index)
plt.xticks(rotation=90)

In [ ]:
basket['Item'].value_counts().iloc[:10]

In [ ]:
plt.xticks(rotation=45)
sns.countplot(x = 'Item', data = basket, order = basket['Item'].value_counts().iloc[:10].index)

In [ ]:
#TEA OR COFFEE?

TC = basket.groupby("Date")["Item"].apply(lambda x: x[x=="Tea"].count()).rename("Teas sold").to_frame()
TC["Coffees sold"] = basket.groupby("Date")["Item"].apply(lambda x: x[x=="Coffee"].count())

TC.plot(figsize=(12,5))
plt.ylim([0,80])
plt.grid(True)
plt.legend()

print("Maxiumum numbers of teas sold: " + str(TC["Teas sold"].max()))
print("Maxiumum numbers of coffees sold: " + str(TC["Coffees sold"].max()))

### 3. *Sales over time* :  <a id="Sale"></a>

In [ ]:
month_year=basket.copy()  
month_year['Date']=pd.to_datetime(month_year['Date'])
month_year['Month'],month_year['Year']=month_year['Date'].dt.month,month_year['Date'].dt.year

grp_month_year=month_year.groupby(['Month','Year'])['Transaction'].count().reset_index()
grp_month_year['Period'] = grp_month_year.Month.astype(str).str.cat(grp_month_year.Year.astype(str), sep=' / ')

#plot graph for each month of the year 2016,2017
fig,axis=plt.subplots(figsize=(5,4))
axis=sns.barplot(data=grp_month_year,x='Period',y='Transaction',color = ("#FC4E07"))
axis.set_xlabel('Month & Year')
axis.set_ylabel('Nombre of Transarctions')
axis.set_xticklabels(grp_month_year['Period'], rotation=60)

In [ ]:
# number of transactions per day
basket["Date"] = pd.to_datetime(basket["Date"])
basket["Weekday"] = basket["Date"].dt.weekday_name
basket['Weekday'].value_counts()

In [ ]:
# plot of items sold per day
products_days = basket["Date"].value_counts().sort_index().rename("Products sold")

mean = round(products_days.mean(),0)
maximum_day = products_days.index[products_days==products_days.max()][0]

ax = products_days.plot(figsize=(12,5), x_compat=True)

text1 = '$\mu=$' + str(mean)[:-2]
# x1 = mdates.date2num(pd.Timestamp("2016/10/31"))
# x2 = mdates.date2num(maximum_day)
# ax.text(x1-5, mean+5, text1, fontsize=15)
# ax.text(x2+1, 300, str(maximum_day)[:-8], fontsize=10)
plt.grid(True)
plt.title("Number of products sold every day")
plt.axhline(mean, c="k", linestyle='--')
plt.tight_layout()

In [ ]:
# items sold per weekday
fig, ax = plt.subplots()
days = ['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday', 'Sunday']
products_weekdays = basket["Weekday"].value_counts().rename("Products sold").reindex(days)
products_weekdays.plot(kind='bar', figsize=(8,7), ax=ax, color=("#00b3b0"))
plt.xticks(rotation=0)
for i, v in enumerate(products_weekdays.values):
    ax.text(i, v+80, str(v), color="#000d1a", fontweight='bold', fontsize=14, ha='center')


In [ ]:
#part of the day groupby
part_of_day=basket.copy()
part_of_day['timestamp'] = part_of_day.Date.astype(str).str.cat(part_of_day.Time.astype(str), sep=' ')
part_of_day['timestamp']=pd.to_datetime(part_of_day['timestamp'])

part_of_day['hour'] = part_of_day['timestamp'].dt.round('H').dt.hour
part_of_day.drop(['Time','Date'],axis=1,inplace=True)

#### Analysing for the best products : Coffee, Tea, Cake, Toast

In [ ]:
#Coffee hours
cof_hours=part_of_day[(part_of_day['Item']== 'Coffee')]
cof_hours=cof_hours.groupby('hour')['Item'].count()
cof_hours=cof_hours.reset_index()
cof_hours

In [ ]:
fig,ax=plt.subplots(figsize=(8,5))
ax=sns.barplot(data=cof_hours,x='hour',y='Item')
ax.set_xlabel('Hours Of The Day')
ax.set_ylabel('Nombre of Times Coffee is Sold')

In [ ]:
#Bread hours
bread_hours=part_of_day[part_of_day['Item']=='Bread']
bread_hours=bread_hours.groupby('hour')['Item'].count()
bread_hours=bread_hours.reset_index()

fig,ax=plt.subplots(figsize=(8,5))
ax=sns.barplot(data=bread_hours,x='hour',y='Item')
ax.set_xlabel('Hours Of The Day')
ax.set_ylabel('Nombre of Times Bread is Sold')

In [ ]:
#Cake or Pastry hours
cake_hours=part_of_day.loc[(part_of_day['Item']=='Cake') | (part_of_day['Item']=='Pastry')]
cake_hours=cake_hours.groupby('hour')['Item'].count()
cake_hours=cake_hours.reset_index()

fig,ax=plt.subplots(figsize=(8,5))
ax=sns.barplot(data=cake_hours,x='hour',y='Item')
ax.set_xlabel('Hours Of The Day')
ax.set_ylabel('Nombre of Times Cake and pastry is Sold')

In [ ]:
#Cake hours
cake_hours=part_of_day.loc[(part_of_day['Item']=='Cake')]
cake_hours=cake_hours.groupby('hour')['Item'].count()
cake_hours=cake_hours.reset_index()

fig,ax=plt.subplots(figsize=(8,5))
ax=sns.barplot(data=cake_hours,x='hour',y='Item')
ax.set_xlabel('Hours Of The Day')
ax.set_ylabel('Nombre of Times Cake is Sold')

In [ ]:
#Tea hours
tea_hours=part_of_day.loc[(part_of_day['Item']=='Tea')]
tea_hours=tea_hours.groupby('hour')['Item'].count()
tea_hours=tea_hours.reset_index()

fig,ax=plt.subplots(figsize=(8,5))
ax=sns.barplot(data=tea_hours,x='hour',y='Item')
ax.set_xlabel('Hours Of The Day')
ax.set_ylabel('Nombre of Times Tea is Sold')

### 4. *Market Basket Analysis* : <a id="MBA"></a>

In [ ]:
df = basket.groupby(['Transaction','Item']).size().reset_index(name='count')
basket0 = (df.groupby(['Transaction', 'Item'])['count']
          .sum().unstack().reset_index().fillna(0)
          .set_index('Transaction'))
#The encoding function
def encode_units(x):
    if x <= 0:
        return 0
    if x >= 1:
        return 1
basket_sets = basket0.applymap(encode_units)

In [ ]:
basket_sets.head()

In [ ]:
basket_sets.corr() 

In [ ]:
# correlation of Churn with other variables
basket_sets.corr()['Coffee'].sort_values(ascending = False).plot(kind='bar', figsize=(20,5), color=("#4951ad"))

In [ ]:
basket_sets.corr()['Coffee'].sort_values(ascending = False)

In [ ]:
basket_sets.corr()['Tea'].sort_values(ascending = False).plot(kind='bar', figsize=(20,5), color=("#7fc94d"))

In [ ]:
basket_sets.corr()['Tea'].sort_values(ascending = False)

### * One Antecedent* <a id="one"></a>

In [ ]:
frequent_itemsets = apriori(basket_sets, min_support=0.01, use_colnames=True)
rules = association_rules(frequent_itemsets, metric="lift")
rules.sort_values('confidence', ascending = False, inplace = True)
print(rules.shape)
rules.head(5)

In [ ]:
# Visualizing the rules distribution color mapped by Lift
plt.figure(figsize=(10, 6))
plt.scatter(rules['support'], rules['confidence'], c=rules['lift'], alpha=0.9, cmap='BuPu');
plt.title('Rules distribution color mapped by lift');
plt.xlabel('Support')
plt.ylabel('Confidence')
plt.colorbar();

In [ ]:
# Set the metric "lift" with a minimum threshold = 1.2

frequent_itemsets = apriori(basket_sets, min_support=0.01, use_colnames=True)
rules2 = association_rules(frequent_itemsets, metric="lift",  min_threshold=1.2)
rules2.sort_values('confidence', ascending = False, inplace = True)
print(rules2.shape)
rules2

In [ ]:
# Visualizing the rules distribution color mapped by Lift
plt.figure(figsize=(10, 6))
plt.scatter(rules2['support'], rules2['confidence'], c=rules2['lift'], alpha=0.9, cmap='BuPu');
plt.title('Rules distribution color mapped by lift');
plt.xlabel('Support')
plt.ylabel('Confidence')
plt.colorbar();

### * More than One antecedent* <a id="two"></a>

In [ ]:
rules["antecedent_len"] = rules["antecedents"].apply(lambda x: len(x))
print(rules.shape)
rules.head(10)

In [ ]:
rules[rules['antecedent_len'] >=2 ]

In [ ]:
# find rules with some conditions :
rules[(rules['antecedent_len'] >= 2) &
      (rules['confidence'] >= 0.1)& 
      (rules['lift'] >= 1.2) ]

### 5 *Conclusion* : <a id="Conc"></a>
The Top Products : Coffee, Bread, Tea, Cake             
The Best time to set a Marketing campaign : Saturday between 11h00-14h00  (*from the output of [Sales over time](#Sale)*) <br>
The Best combination :<br>

[1. One Item Antecedente :](#one) <br>
Toast => ** Coffee ** <br>
- The lift of an association rule “if Toast then Coffee” is 1.48 <br>
- The confidence is 70%. <br>

Spanish Brunch => ** Coffee ** <br>
- The lift of an association rule “if Spanish Brunch then Coffee” is 1.26 <br>
- The confidence is 59%. <br>

Cake => ** Tea ** <br>
- The lift of an association rule "if Cake then  Tea " is 1.61 
- The confidence is 22%. <br>

[2. More than One Item Antecedente :](#two) <br>
Coffee, Bread => Pastry <br>
- The lift of an association rule "if Coffee and Bread then  Pastry " is 1.45 <br> 
- The confidence is 12%. <br>

(*from the output of [Market Basket Analysis](#MBA)*)

### 6. *References :* <a id="Book"></a>
* Database Marketing_ Analyzing and Managing Customers-Springer (2008) Robert C. Blattberg, Byung-Do Kim, Scott A. Neslin - (International Series in Quantitative Marketing) - https://www.springer.com/gp/book/9780387725789

* Apriori Algorithm: http://rasbt.github.io/mlxtend/user_guide/frequent_patterns/apriori/